In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import matplotlib.animation
import pandas as pd
from IPython.display import HTML

sns.set_style("darkgrid")
plt.rcParams["animation.html"] = "html5"

In [2]:
class Graph(nx.Graph):
    def random_graph(self, n, p):
        eges_nr = np.sum([i for i in range(n)])
        edges = np.array([[i, j] for i in range(n-1) for j in range(i+1,n) if np.random.uniform() < p ])
        # all_edges = np.array([[i, j] for i in range(n-1) for j in range(i+1,n)])
        # taked_edes = np.random.uniform(size=eges_nr)<p
        # edges_to_add = all_edges[taked_edes]
        self.add_nodes_from(range(n))
        # self.add_edges_from(edges_to_add)
        self.add_edges_from(edges)
        return self
    
    def Watts_Strogatz_model(self, n , k, p):
        """Returns a Watts–Strogatz small-world graph.

        Parameters
        ----------
        n : int
            The number of nodes
        k : int
            Each node is joined with its `k` nearest neighbors in a ring
            topology.
        p : float
            The probability of rewiring each edge
        """
        if k>n:
            raise ValueError(f"The parameter 'k' = {k} is greater than parametr 'n' = {n}")
        elif k == n:
            return nx.complete_graph(n)
        # g = nx.Graph()
        self.add_nodes_from(range(n))
        
        nodes = self.nodes
        for i in range(n):
            for j in range(1, k // 2 + 1):
                self.add_edge(i, (i+j)%n)
                
        for node in nodes:
            for neighbour in range(1, k // 2 + 1):
                if np.random.uniform() < p:
                    random_note = np.random.choice(nodes)
                    while node == random_note or self.has_edge(node, random_note):
                        random_note = np.random.choice(nodes)
                    else:
                        self.remove_edge(node, (node + neighbour)%n)
                        self.add_edge(node, random_note)
                else:
                    pass
        return self

    def barabasi_albert_model(self, m0, n, iterations, inicial_praph = None):
        if inicial_praph:
            g = inicial_praph
        else:
            g = nx.complete_graph(m0)
        degrees = {note:degre for note,degre in g.degree() }
        for node in range(m0, m0 + iterations):
            values = np.array(list(degrees.values()))
            nodes_to_conect = np.random.choice(list(degrees.keys()), size = n, replace=False, p = values/sum(values))
            degrees[node] = n
            for node_to_conect in nodes_to_conect:
                degrees[node_to_conect] += 1
            g.add_edges_from([[node, i] for i in nodes_to_conect ])
        return g
        
    def lattice_2d(m, n):
        g = nx.grid_2d_graph(m, n)
        return g

In [3]:
def q_voter_model(graph, p, q, epsilon, MC_steps, graph_name = None, set_attributes = True, save = True) :
    if set_attributes:
        nx.set_node_attributes(graph, 1, "state")
    nodes = graph.nodes()
    N = len(nodes)
    save_array_name = f"graph_{graph_name} p_{p} q_{q} epsilon_{epsilon} MC_steps_{MC_steps}"
    save_array = np.zeros((MC_steps, N))

    nodes_list = np.arange(0, N , 1)
    attributes_dict = nx.get_node_attributes(graph, "state")
    attributes_list = np.array(list(attributes_dict.values()))
    neighbors_list = tuple(tuple(graph.neighbors(n)) for n in nodes)
    for MC_step in range(MC_steps):
        random_spinson = np.random.randint(0, N, N)
        #independence case
        independence = np.random.uniform(size=N) < p
        independence_spinson = random_spinson[independence]
        attributes_list[independence_spinson]  *= -1
        #other case
        not_independence_spinson = random_spinson[~independence]
        random_neighbors = [random.choices(neighbors_list[n], k = q) for n in not_independence_spinson]
        random_neighbors_state = attributes_list[random_neighbors]
        same_state = np.abs(np.sum(random_neighbors_state, 1)) == q
        attributes_list[not_independence_spinson[same_state]] *= -1
        #otherwise
        not_unanimous = not_independence_spinson[~same_state]
        flip_not_unanimous = not_unanimous[np.random.uniform(size=len(not_unanimous)) < epsilon]
        attributes_list[flip_not_unanimous] *= -1
        save_array[MC_step] = attributes_list
    if save:
        np.save(f"save_array/{save_array_name}", save_array)
    return save_array

In [15]:
m0 = 2
n = 2
iterations  = 98
BA = Graph().barabasi_albert_model(m0, n, iterations = iterations)

In [5]:
p = 0.5
q = 3
MC_steps = 10**6
epsilon = 0.1
graph_name = "BA"
save_array_name = f"graph_{graph_name} p_{p} q_{q} epsilon_{epsilon} MC_steps_{MC_steps}"
q_voter_model(BA, p, q, epsilon, MC_steps, graph_name)

In [6]:
# with open(f"save_array/{save_array_name}.npy", 'rb') as f:
with open(f"save_array/{save_array_name}.npy", 'rb') as f:
    array = np.load(f)
array

array([[-1.,  1., -1., ...,  1., -1., -1.],
       [ 1.,  1.,  1., ...,  1.,  1., -1.],
       [-1.,  1.,  1., ...,  1., -1., -1.],
       ...,
       [-1.,  1.,  1., ...,  1.,  1., -1.],
       [ 1., -1.,  1., ...,  1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]])

In [7]:
array.shape

(1000000, 100)

In [ ]:
m0 = 2
n = 2
iterations  = 98
graph = Graph().barabasi_albert_model(m0, n, iterations = iterations)
graph_name = "BA"
p = 0.5
epsilon = 0.5
q = 3
MC_steps = 2


nx.set_node_attributes(graph, 1, "state")
nodes = graph.nodes()
N = len(nodes)
save_array_name = f"graph_{graph_name} p_{p} q_{q} epsilon_{epsilon} MC_steps_{epsilon}"
save_array = np.zeros((MC_steps, N))

nodes_list = np.arange(0, N , 1)
attributes_dict = nx.get_node_attributes(graph, "state")
attributes_list = np.array(list(attributes_dict.values()))
neighbors_list = tuple(tuple(graph.neighbors(n)) for n in nodes)
for MC_step in range(MC_steps):
    random_spinson = np.random.randint(0, N, N)
    #independence case
    independence = np.random.uniform(size=N) < p
    independence_spinson = random_spinson[independence]
    attributes_list[independence_spinson]  *= -1
    #other case
    not_independence_spinson = random_spinson[~independence]
    random_neighbors = [random.choices(neighbors_list[n], k = q) for n in not_independence_spinson]
    random_neighbors_state = attributes_list[random_neighbors]
    same_state = np.abs(np.sum(random_neighbors_state, 1)) == q
    attributes_list[not_independence_spinson[same_state]] *= -1
    #otherwise
    not_unanimous = not_independence_spinson[~same_state]
    flip_not_unanimous = not_unanimous[np.random.uniform(size=len(not_unanimous)) < epsilon]
    attributes_list[flip_not_unanimous] *= -1
    save_array[MC_step] = attributes_list
np.save(f"save_array/{save_array_name}",save_array)